Analisis casos manuales

In [ ]:
import os
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import MonthLocator, YearLocator

username = os.getlogin()
hoy = datetime.datetime.now()
date_formatcarga = "%d.%m.%Y"
date_formatexport = "%d-%m-%Y"
date_formatcasos = "%d-%m-%Y %H:%M"
date_formatpiem = "%Y-%m"

fecha_actual = hoy.strftime(date_formatcarga)
df1 = pd.DataFrame
file_path = f'C:\\Users\\{username}\\Documents\\Py\\CASOS\\tablacasos.xlsx'
df1 = pd.read_excel(file_path,
                    sheet_name='Historial casos Manuales',header=0,
                    names=['caso','agencia','tdep','contacto','asunto','cierre','montocierre','depositado','banco','user','fapertura','fmod','alias','usermod'])
df1['tdep'] = df1['tdep'].astype(str)
df1.fillna('null', inplace=True)

df1['fapertura'] = pd.to_datetime(df1['fapertura'],format = date_formatcasos)
df1['fmod'] = pd.to_datetime(df1['fmod'],format = date_formatexport)

print("Tabla cargada.")

Obtener la cantidad de casos cerrados

In [ ]:
count_df1 = df1[df1['usermod'].str.contains('ioliv')].groupby('usermod').size().reset_index(name='count')
count_df2 = df1[df1['usermod'].str.contains('procom')].groupby('usermod').size().reset_index(name='count')
count_df3 = df1[df1['usermod'].str.contains('sysadmin')].groupby('usermod').size().reset_index(name='count')
count_df = pd.concat([count_df1,count_df2,count_df3])
sorted_df2 = count_df2
print(count_df)
print(f"Total casos operador: {(sum(count_df['count']))}")                                                            

Obtener cantidad de casos creados por usuario

In [ ]:
count_cpu= df1.groupby('user').size().reset_index(name='count').sort_values(by='count', ascending=False)
count_cpu = count_cpu.set_index('user')
count_cpu = count_cpu[count_cpu['count'] > 50]
count_cpu.plot.bar(title='Casos por usuario',xlabel='Mes',ylabel='Casos',figsize=(17,4))
#plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, p:f'{int(x/1000000):,}'))
plt.xticks(rotation=30, ha='right')
plt.grid(axis='y')
plt.yticks(np.arange(0,9000,step=500))
plt.show()

In [ ]:
dfc = df1.copy()
count_df = dfc.loc[dfc['fapertura'] > '2024-01-01']
count_df = count_df[count_df['tdep'].str.contains('Saldo de Ajuste')].groupby('tdep').size().reset_index(name='count')
sorted_df = count_df.sort_values(by='count', ascending=False)
print (sorted_df)
#sorted_df.to_csv('C:\\Users\\dxs81864\\Documents\\Py\\CASOS\\GENERADOS\\Pdc ' + fecha_actual + '.csv',
#               sep=';',
#               header=False,
#               index=False)
print(f"Total casos: {(sum(count_df['count']))}")  
                                  

Obtener la ocurrencia de tipo de caso

Reemplazar tipo de caso y periodicidad

In [ ]:
fechadesde = '2024-01-01'
periodicidad = 'M'
percasos = df1.copy()
percasos = percasos.loc[percasos['fapertura'] > fechadesde]
percasos = percasos.groupby(df1['fapertura'].dt.to_period(periodicidad))['tdep'].count()
countcasos = percasos.resample(periodicidad).sum()
#print(count)
print(countcasos.mean())

In [ ]:

#countcasos.index = countcasos.index.map(str)
#countcasos.index.astype(str, copy = False)
#print(countcasos.index)
#countcasos.index = pd.to_datetime(countcasos.index.astype(str),format='format')
countcasos.plot.bar(title='Casos',xlabel='Mes',ylabel='Casos',figsize=(10,4))
#plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, p:f'{int(x/1000000):,}'))

years = np.arange(2022,2025)
plt.xticks(rotation=30, ha='right')
yloc = YearLocator()
mloc = MonthLocator()
#plt.gca().xaxis.set_major_locator(yloc)
#plt.gca().xaxis.set_minor_locator(mloc)
plt.grid(axis='y',which='major')
plt.yticks(np.arange(0,2000,step=100))
plt.show()
print(countcasos.mean())

In [ ]:
dfbar = df1.copy()
fechadesde = '2024-01-01'
dfbar = dfbar.loc[dfbar['fapertura'] > fechadesde]
dfbar['month'] = dfbar['fapertura'].dt.month
dfbar['year'] = dfbar['fapertura'].dt.year
count_dfbar = dfbar.groupby(['tdep','year','month']).size().reset_index(name='count')
count_dfbar['fecha'] = count_dfbar['year'].astype(str).str.zfill(2) + '-' + count_dfbar['month'].astype(str).str.zfill(2)
count_dfbar = count_dfbar.drop(['year','month'],axis=1)
count_dfbar = count_dfbar.pivot_table(index='fecha',columns='tdep',values='count',aggfunc='sum')
count_dfbar = count_dfbar.drop(columns=['Parcializado'])


count_dfbar.plot.bar(title='Casos según tipo',xlabel='Mes',ylabel='Casos', stacked=True, figsize=(14,7))

#plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, p:f'{int(x/1000000):,}'))
plt.xticks(rotation=30, ha='right')
plt.grid(axis='y')
plt.yticks(np.arange(0,1400,step=100))
plt.show()


In [ ]:
tipocaso = 'Parcializado'
fechadesde = '2024-01-01'
periodicidad = 'M'
periodicity = df1[df1['tdep'].str.contains(tipocaso)]
periodicity = periodicity.loc[periodicity['fapertura'] > fechadesde]
periodicity = periodicity.groupby(df1['fapertura'].dt.to_period(periodicidad))['tdep'].count()
countparc = periodicity.resample(periodicidad).sum()
#print(count)
print(countparc.mean())



In [ ]:
countparc.plot.bar(title='Casos Parcializados',xlabel='Mes',ylabel='Casos',figsize=(14,4))
#plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, p:f'{int(x/1000000):,}'))
plt.xticks(rotation=30, ha='right')
plt.grid(axis='y')
#plt.yticks(np.arange(-40000000,170000000,step=10000000))
plt.show()

In [ ]:
tipocaso = 'Pago Saldo Ajuste'
fechadesde = '2024-01-01'
periodicidad = 'M'
periodicitysa = df1[df1['tdep'].str.contains(tipocaso)]
periodicitysa = periodicitysa.loc[periodicitysa['fapertura'] > fechadesde]
periodicitysa = periodicitysa.groupby(df1['fapertura'].dt.to_period(periodicidad))['tdep'].count()
countsa = periodicitysa.resample(periodicidad).sum()
#print(count)
print(countsa.mean())

In [ ]:
countsa.plot.bar(title='Casos Saldo de Ajuste',xlabel='Mes',ylabel='Casos',figsize=(15,4))
#plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, p:f'{int(x/1000000):,}'))
plt.xticks(rotation=30, ha='right')
plt.grid(axis='y')
plt.yticks(np.arange(0,90,step=10))
plt.show()

Casos por alias

In [32]:
dfalias = df1[df1['user'].str.contains('autocase')]

Obtener ocurrencia por agencia de casos parcializados por agencia agrupado según mes

In [ ]:
df_parc = df1.loc[df1['fapertura'] > '2024-01-01']
df_parc = df_parc[df_parc['agencia'] != "809000"]
df_parc = df_parc[df_parc['agencia'] != "801000"]
df_conteoparc = df_parc.groupby([pd.Grouper(key='agencia')]).size().reset_index(name='periodicity')
df_conteoparc = df_conteoparc.sort_values(by=['periodicity','agencia'], ascending=[False,False])
#df_conteoparc.drop_duplicates(subset='agencia',keep='first', inplace=True)
print(df_conteoparc)


Agencias Parcializados

In [ ]:
df_conteoparcmuestra = df_conteoparc[df_conteoparc['periodicity'] > 100].set_index('agencia')

df_conteoparcmuestra.plot.bar(title='Casos por agencia',xlabel='Agencias',ylabel='Casos',figsize=(15,2))
#plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, p:f'{int(x/1000000):,}'))
plt.xticks(rotation=20, ha='right')
plt.grid(axis='y')
plt.yticks(np.arange(0,410,step=50))
plt.show()

In [62]:
df_conteoparctotal = df_conteoparc.groupby('agencia')['periodicity'].sum().reset_index()
df_conteoparctotal = df_conteoparctotal.sort_values(by='periodicity', ascending=False)

Tabla de casos pre-conciliados

In [76]:
dfpc = df1.copy()

dfpc = dfpc[~dfpc['asunto'].str.contains('B2B')]
dfpc = dfpc[~dfpc['tdep'].str.contains('S.com')]
dfpc = dfpc.loc[dfpc['fapertura'] > '2024-01-01']
df_period = dfpc[dfpc['usermod'].str.contains('') | dfpc['usermod'].str.contains('')]

conteo_preconc = df_period.groupby(dfpc['fapertura'].dt.to_period('M'))['caso'].count()
#conteo_parc = df[dfc['tdep'].str.contains('Parcializado')].groupby(dfc['fapertura'].dt.to_period('M'))['caso'].count()
#conteo_pc = dfc[dfc['tdep'].str.contains('Problema cartola')].groupby(dfc['fapertura'].dt.to_period('M'))['caso'].count()

dfpreconc_plot = conteo_preconc.copy().resample('M').sum()
#dfparc_plot = conteo_parc.copy().resample('M').sum()
#dfpc_plot = conteo_pc.copy().resample('M').sum()

C:\Users\dxs81864\AppData\Local\Temp\ipykernel_11060\3860703131.py:12: FutureWarning: Resampling with a PeriodIndex is deprecated. Cast index to DatetimeIndex before resampling instead.
  dfpreconc_plot = conteo_preconc.copy().resample('M').sum()


In [ ]:
dfpreconc_plot.plot(kind='bar',x='count', figsize=(10,4))
plt.title('Casos Pre-conciliados')
plt.xlabel('Periodo')
plt.xticks(rotation=30, ha='right')
plt.ylabel('Casos', fontsize = 14)
plt.yticks(np.arange(0,750,step=50))
plt.grid(axis='y')
#for i, v in enumerate(y):
#    plt.text(xlocs[i] - 0.25, v + 0.01, str(v))
#myFmt = mdates.DateFormatter('%b/%Y')
#plt.gca().xaxis.set_major_formatter(myFmt)
#plt.xticks(np.arange('2023-12', '2024-12', dtype='datetime64[M]'),rotation=10, ha='right')
plt.show()

In [ ]:
dfpc_plot.plot(kind='bar',x='count', figsize=(15,4))
plt.title('Casos Problema Cartola')
plt.xlabel('Periodo')
plt.xticks(rotation=30, ha='right')
plt.ylabel('Casos', fontsize = 14)
plt.yticks(np.arange(0,550,step=50))
plt.grid(axis='y')
arrcasos = dfpc_plot.to_numpy()